This notebook shows the integration of EXIOBASE into brightway2 as aggregated results. In other words, EXIOBASE will only be available as aggregated LCIA scores.

In [1]:
# all modules can be imported with pip, outside of brightway2
import pandas as pd
import pymrio
import hashlib
from brightway2 import *
from bw2agg.scores import add_all_unit_score_exchanges_and_cfs

Select (or create) the brightway2 project you wish to integrated EXIOBASE into

In [9]:
projects.set_current('Importing EXIOBASE')

Need to setup brightway2:

In [ ]:
bw2setup()

Need to download IMPACT World+ from https://zenodo.org/record/3521034 and load it into brightway:

In [ ]:
path_to_IW_package = '/.../Brightway_IW_damage_1_46_and_midpoint_1_28.bw2package'
BW2Package.import_file(path_to_IW_package)

Load EXIOBASE3 (for EXIOBASE2, use the already existing importer of brightway2) using pymrio and define normalized matrices using .calc_all()

In [2]:
path_exiobase = 'path_to_exiobase_folder'
io = pymrio.parse_exiobase3(path_exiobase)
io.calc_all()

EXIOBASE3 includes numbers in some of the sectors, we just swiftly remove them:

In [3]:
sectors_without_numbers = []
for sector in list(io.get_sectors()):
    if any(char.isdigit() for char in sector):
        sectors_without_numbers.append(sector[:-5])
    else:
        sectors_without_numbers.append(sector)
index_without_numbers = pd.MultiIndex.from_product([list(io.get_regions()),sectors_without_numbers])

io.A.index = index_without_numbers
io.A.columns = index_without_numbers
io.L.index = index_without_numbers
io.L.columns = index_without_numbers
io.satellite.S.columns = index_without_numbers

We give hashcodes to EXIOBASE sectors to follow how brightway2 typically deals with unique identifiers.

In [5]:
# create hash codes for exiobase sectors
give_hash_to_exio = dict.fromkeys(io.A.index)
give_hash_to_exio = {k:hashlib.md5(str(k).encode()).hexdigest() for k,v in give_hash_to_exio.items()}
# set those hash codes as index
io.A.index = [give_hash_to_exio[i] for i in io.A.index]
io.A.columns = [give_hash_to_exio[i] for i in io.A.columns]
io.L.index = [give_hash_to_exio[i] for i in io.L.index]
io.L.columns = [give_hash_to_exio[i] for i in io.L.columns]
io.satellite.S.columns = [give_hash_to_exio[i] for i in io.satellite.S.columns]

Pymrio does not include characterization matrices. We fetch it. Use it to aggregate results of EXIOBASE and link charcaterization methods names to the ones used by brightway2.

In [32]:
# load IW+ characterization factors for exiobase (available here: https://zenodo.org/record/3955079)
C = pd.read_csv('path_to_csv_file')

# reindex to have a name dataframe
C.set_index('Unnamed: 0',drop=True,inplace=True)

# aggregate results of exiobase
aggregated_results = C.dot(io.satellite.S.dot(io.L).iloc[:-9])/1000000

# link IMPACT World+ impact categories typology to the methods abbreviations used in brightway2
IW_pylcaio_to_bw2 = dict.fromkeys(C.index)

for IW_category in IW_pylcaio_to_bw2:
    if ('PDF' in IW_category or 'DALY' in IW_category):
        IW_pylcaio_to_bw2[IW_category] = methods.get(('IMPACTWorld+ (Default_Recommended_Damage 1.46)', IW_category.split(' (')[0]))['abbreviation']
    else:
        IW_pylcaio_to_bw2[IW_category] = methods.get(('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)', IW_category.split(' (')[0]))['abbreviation']

# define these abbreviations as index for C
aggregated_results.index = list(IW_pylcaio_to_bw2.values())

To operate with aggregated data, we need to use functionality from bw2agg. Here, dummy exchanges and characterization factors are create to enable the storing of LCIA scores into exchanges directly.

In [34]:
add_all_unit_score_exchanges_and_cfs()

Adding unit score biosphere exchanges and characterization factors to all 895 methods in project


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:06:41


We change the dataframe of exchanges into the dictionary format used by brightway2

In [41]:
bw2_dict = dict.fromkeys(list(zip(['exiobase']*len(aggregated_results.columns),aggregated_results.columns)),{
    'name': '',
    'unit':'',
    'location':'',
    'exchanges':[],
})

for sector_hash in aggregated_results.columns:
    bw2_dict[('exiobase',sector_hash)] = {'name' : list({k:v for k,v in give_hash_to_exio.items() if v == sector_hash}.keys())[0][1],
                                          'unit' : 'euro',
                                          'location' : list({k:v for k,v in give_hash_to_exio.items() if v == sector_hash}.keys())[0][0],
                                          'exchanges': aggregated_results.loc[:, sector_hash][aggregated_results.loc[:,sector_hash]!=0].to_dict()}
for sector_hash in aggregated_results.columns:
    bw2_dict[('exiobase',sector_hash)]['exchanges'] = list({k:{'input':('biosphere3',k), 'amount':v, 'type':'biosphere'} for k,v in bw2_dict[('exiobase',sector_hash)]['exchanges'].items()}.values())

Load the dictionary created as a database (whose name you can choose) of brightway2

In [44]:
Database('exiobase').write(bw2_dict)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:06:13


Title: Writing activities to SQLite3 database:
  Started: 08/04/2020 16:18:14
  Finished: 08/04/2020 16:24:27
  Total time elapsed: 00:06:13
  CPU %: 10.50
  Memory %: 9.72


Finished, EXIOBASE is integrated into your brightway2 project.